In [0]:

%python
#Mount Azure Blob Storage

# Fetching the csv file from the blob storage
storage_account_name = "sourceaccount1hexa"
container_name = "superstore-data"
storage_account_key = "ooZRQBCJR9AwKPvv5YvKprjv3UlGndcovcyJcjT4dVN8DbxojxsSgLojGldlb4O0ZMJd1AqFqrY2+AStj31gDQ=="

# Unmount the directory if it is already mounted
if any(mount.mountPoint == "/mnt/superstore" for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount("/mnt/superstore")

# Mount Blob Storage
dbutils.fs.mount(
    source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point="/mnt/superstore",
    extra_configs={
        f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key
    }
)

/mnt/superstore has been unmounted.


True

In [0]:
#verifying the mount

display(dbutils.fs.ls("/mnt/superstore"))

path,name,size,modificationTime
dbfs:/mnt/superstore/Global_Superstore2.csv,Global_Superstore2.csv,12089916,1734265964000


In [0]:
# Load the dataset
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SuperstoreETL").getOrCreate()

file_path = "/mnt/superstore/Global_Superstore2.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show(5)


+------+---------------+----------+----------+------------+-----------+----------------+-----------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+
|Row ID|       Order ID|Order Date| Ship Date|   Ship Mode|Customer ID|   Customer Name|    Segment|         City|          State|      Country|Postal Code|Market| Region|      Product ID|  Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|Shipping Cost|Order Priority|
+------+---------------+----------+----------+------------+-----------+----------------+-----------+-------------+---------------+-------------+-----------+------+-------+----------------+----------+------------+--------------------+--------+--------+--------+--------+-------------+--------------+
| 32298| CA-2012-124891|2012-07-31|2012-07-31|    Same Day|   RH-19495|     Rick Hansen|   Consumer|New

In [0]:
# Inspect and Verify Schema
df.printSchema()


root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Market: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Shipping Cost: double (nullable = true)
 |-- Order Priority: string (nullable = true)



In [0]:
#Persist Raw Data (Bronze Zone)
bronze_zone_path = "/mnt/bronze/superstore"
df.write.format("delta").option("mergeSchema", "true").option("delta.columnMapping.mode", "name").mode("overwrite").save(bronze_zone_path)


In [0]:
#Perform Data Validation Checks
# Check for null values in critical columns

from pyspark.sql.functions import col
df.filter(col("Order ID").isNull() | col("Customer ID").isNull()).show()



+------+--------+----------+---------+---------+-----------+-------------+-------+----+-----+-------+-----------+------+------+----------+--------+------------+------------+-----+--------+--------+------+-------------+--------------+
|Row ID|Order ID|Order Date|Ship Date|Ship Mode|Customer ID|Customer Name|Segment|City|State|Country|Postal Code|Market|Region|Product ID|Category|Sub-Category|Product Name|Sales|Quantity|Discount|Profit|Shipping Cost|Order Priority|
+------+--------+----------+---------+---------+-----------+-------------+-------+----+-----+-------+-----------+------+------+----------+--------+------------+------------+-----+--------+--------+------+-------------+--------------+
+------+--------+----------+---------+---------+-----------+-------------+-------+----+-----+-------+-----------+------+------+----------+--------+------------+------------+-----+--------+--------+------+-------------+--------------+



In [0]:
#Deduplicate Raw Data
df = df.dropDuplicates(["Order ID", "Row ID"])


In [0]:
#Save Cleaned Raw Data in Bronze Zone
df.write.format("delta").mode("overwrite").save(bronze_zone_path)


In [0]:
dbutils.notebook.exit(bronze_zone_path)
